-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Hyperparameter Tuning with Random Forests

In this lab, you will convert the Airbnb problem to a classification dataset, build a random forest classifier, and tune some hyperparameters of the random forest.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Learning Objectives:<br>

By the end of this lab, you should be able to;

* Perform grid search on a random forest based model
* Generate feature importance scores and classification metrics for a random forest model
* Identify differences between scikit-learn's and Spark ML's Random Forest implementation


 
You can read more about the distributed implementation of Random Forests in the Spark <a href="https://github.com/apache/spark/blob/master/mllib/src/main/scala/org/apache/spark/ml/tree/impl/RandomForest.scala#L42" target="_blank">source code</a>.

## Lab Setup

The first thing we're going to do is to **run setup script**. This script will define the required configuration variables that are scoped to each user.

In [0]:
%run "../Includes/Classroom-Setup"

Python interpreter will be restarted.
Python interpreter will be restarted.


Resetting the learning environment:
| No action taken

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/scalable-machine-learning-with-apache-spark/v02"

Validating the locally installed datasets:
| listing local files...(2 seconds)
| validation completed...(2 seconds total)

Creating & using the schema "odl_user_1002406_0svy_da_sml" in the catalog "hive_metastore"...(1 seconds)

Predefined tables in "odl_user_1002406_0svy_da_sml":
| -none-

Predefined paths variables:
| DA.paths.working_dir: dbfs:/mnt/dbacademy-users/odl_user_1002406@databrickslabs.com/scalable-machine-learning-with-apache-spark
| DA.paths.user_db:     dbfs:/mnt/dbacademy-users/odl_user_1002406@databrickslabs.com/scalable-machine-learning-with-apache-spark/database.db
| DA.paths.datasets:    dbfs:/mnt/dbacademy-datasets/scalable-machine-learning-with-apache-spark/v02

Setup completed (9 seconds)


## From Regression to Classification

In this case, we'll turn the Airbnb housing dataset into a classification problem to **classify between high and low price listings.**  Our **`class`** column will be:<br><br>

- **`0`** for a low cost listing of under $150
- **`1`** for a high cost listing of $150 or more

In [0]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

file_path = f"{DA.paths.datasets}/airbnb/sf-listings/sf-listings-2019-03-06-clean.delta/"

airbnb_df = (spark
            .read
            .format("delta")
            .load(file_path)
            .withColumn("priceClass", (col("price") >= 150).cast("int"))
            .drop("price")
           )

train_df, test_df = airbnb_df.randomSplit([.8, .2], seed=42)

categorical_cols = [field for (field, dataType) in train_df.dtypes if dataType == "string"]
index_output_cols = [x + "Index" for x in categorical_cols]

string_indexer = StringIndexer(inputCols=categorical_cols, outputCols=index_output_cols, handleInvalid="skip")

numeric_cols = [field for (field, dataType) in train_df.dtypes if ((dataType == "double") & (field != "priceClass"))]
assembler_inputs = index_output_cols + numeric_cols
vec_assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

### Why can't we OHE?

**Question:** What would go wrong if we One Hot Encoded our variables before passing them into the random forest?

**HINT:** Think about what would happen to the "randomness" of feature selection.

## Random Forest

Create a Random Forest classifer called **`rf`** with the **`labelCol=priceClass`**, **`maxBins=40`**, and **`seed=42`** (for reproducibility).

It's under **`pyspark.ml.classification.RandomForestClassifier`** in Python.

In [0]:
# TODO
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="priceClass", maxBins=40, seed=42)
stages =[string_indexer,vec_assembler,rf]
pipeline = Pipeline(stages=stages)

In [0]:
print(rf.explainParams)

<bound method Params.explainParams of RandomForestClassifier_22accd7f09d3>


## Grid Search

There are a lot of hyperparameters we could tune, and it would take a long time to manually configure.

Let's use Spark's <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.tuning.ParamGridBuilder.html?highlight=paramgrid#pyspark.ml.tuning.ParamGridBuilder" target="_blank">ParamGridBuilder</a> to find the optimal hyperparameters in a more systematic approach. Call this variable **`param_grid`**.

Let's define a grid of hyperparameters to test:
  - maxDepth: max depth of the decision tree (Use the values **`2, 5, 10`**)
  - numTrees: number of decision trees (Use the values **`10, 20, 100`**)

**`addGrid()`** accepts the name of the parameter (e.g. **`rf.maxDepth`**), and a list of the possible values (e.g. **`[2, 5, 10]`**).

In [0]:
# TODO

from pyspark.ml.tuning import ParamGridBuilder

parm_grid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [2,5,10])
              .addGrid(rf.numTrees, [10,20,100])
              .build())

## Evaluator

In the past, we used a **`RegressionEvaluator`**.  For classification, we can use a <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.BinaryClassificationEvaluator.html?highlight=binaryclass#pyspark.ml.evaluation.BinaryClassificationEvaluator" target="_blank">BinaryClassificationEvaluator</a> if we have two classes or <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html?highlight=multiclass#pyspark.ml.evaluation.MulticlassClassificationEvaluator" target="_blank">MulticlassClassificationEvaluator</a> for more than two classes.

Create a **`BinaryClassificationEvaluator`** with **`areaUnderROC`** as the metric.

<img src="https://files.training.databricks.com/images/icon_note_24.png"/> <a href="https://en.wikipedia.org/wiki/Receiver_operating_characteristic" target="_blank">Read more on ROC curves here.</a>  In essence, it compares true positive and false positives.

In [0]:
# TODO
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol="priceClass", metricName="areaUnderROC")

## Cross Validation

We are going to do 3-Fold cross-validation and set the **`seed`**=42 on the cross-validator for reproducibility.

Put the Random Forest in the CV to speed up the <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.tuning.CrossValidator.html?highlight=crossvalidator#pyspark.ml.tuning.CrossValidator" target="_blank">cross validation</a> (as opposed to the pipeline in the CV).

In [0]:
# TODO
from pyspark.ml.tuning import CrossValidator

cv = CrossValidator(estimator=rf, evaluator=evaluator, estimatorParamMaps=parm_grid,numFolds=3,seed=42)

## Pipeline

Let's fit the pipeline with our cross validator to our training data (this may take a few minutes).

In [0]:
stages = [string_indexer, vec_assembler, cv]

pipeline = Pipeline(stages=stages)

pipeline_model = pipeline.fit(train_df)



## Hyperparameter

Which hyperparameter combination performed the best?

In [0]:
cv_model = pipeline_model.stages[-1]
rf_model = cv_model.bestModel

list(zip(cv_model.getEstimatorParamMaps(), cv_model.avgMetrics))

#print(rf_model.explainParams())

Out[24]: [({Param(parent='RandomForestClassifier_22accd7f09d3', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 2,
   Param(parent='RandomForestClassifier_22accd7f09d3', name='numTrees', doc='Number of trees to train (>= 1).'): 10},
  0.8494609892340327),
 ({Param(parent='RandomForestClassifier_22accd7f09d3', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 2,
   Param(parent='RandomForestClassifier_22accd7f09d3', name='numTrees', doc='Number of trees to train (>= 1).'): 20},
  0.8450403538026398),
 ({Param(parent='RandomForestClassifier_22accd7f09d3', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 2,
   Param(parent='RandomForestClassifier_22a

## Feature Importance

In [0]:
import pandas as pd

pandas_df = pd.DataFrame(list(zip(vec_assembler.getInputCols(), rf_model.featureImportances)), columns=["feature", "importance"])
top_features = pandas_df.sort_values(["importance"], ascending=False)
top_features

,feature,importance
12,bedrooms,0.156562
5,room_typeIndex,0.145999
10,accommodates,0.145594
3,neighbourhood_cleansedIndex,0.094439
13,beds,0.074897
7,host_total_listings_count,0.056620
8,latitude,0.051551
9,longitude,0.039338
16,review_scores_rating,0.033169
15,number_of_reviews,0.031997


Do those features make sense? Would you use those features when picking an Airbnb rental?

## Apply Model to Test Set

In [0]:
# TODO

pred_df = pipeline_model.transform(test_df)
area_under_roc = evaluator.evaluate(pred_df)

print(f"Area under ROC is {area_under_roc:.2f}")

Area under ROC is 0.92


## Save Model

Save the model to **`DA.paths.working_dir`** (variable defined in Classroom-Setup)

In [0]:
pipeline.model.write().overwrite().save(DA.paths.working_dir)

## Sklearn vs SparkML

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html" target="_blank">Sklearn RandomForestRegressor</a> vs <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.RandomForestRegressor.html?highlight=randomfore#pyspark.ml.regression.RandomForestRegressor" target="_blank">SparkML RandomForestRegressor</a>.

Look at these params in particular:
* **n_estimators** (sklearn) vs **numTrees** (SparkML)
* **max_depth** (sklearn) vs **maxDepth** (SparkML)
* **max_features** (sklearn) vs **featureSubsetStrategy** (SparkML)
* **maxBins** (SparkML only)

What do you notice that is different?

camelCase for Spark
no maxbins for sparkml.

## Classroom Cleanup

Run the following cell to remove lessons-specific assets created during this lesson:

In [0]:
DA.cleanup()

-sandbox
&copy; 2023 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>